In [ ]:
import xarray as xr
from esdl import Cube
import numpy.ma as ma
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import warnings
warnings.filterwarnings('ignore')

In [ ]:
MERCATOR_PROJECTION = dict(projection='merc',llcrnrlat=-60,urcrnrlat=80, llcrnrlon=-180,urcrnrlon=180,lat_ts=20,resolution='c')
ECKERT4_PROJECTION = dict(projection='eck4',lon_0=0,resolution='c')
KAVRAYSKIY7_PROJECTION = dict(projection = "kav7", lon_0=0., resolution=None)

In [ ]:
country_cube_dataset = xr.open_dataset("/home/jovyan/work/datacube/country-cube-0.1.0/country_cube.nc")
country_cube_array = country_cube_dataset.to_array(dim="Indicator_Codes")
ESDC = xr.open_zarr("/home/jovyan/work/datacube/esdc-8d-0.25deg-1x720x1440-1.0.1_1_zarr")
country_code_map = "/home/jovyan/work/datacube/country-cube-0.1.0/country-codes.csv"

In [ ]:
def plot_variable_2d(variable_name, year, title, projection = KAVRAYSKIY7_PROJECTION, save_image = False, **kwargs):
    selected_var_array = country_cube_array.sel(Indicator_Codes = variable_name,Year=year)
    _map_plot_ccube(selected_var_array, title, proj = projection, save_me=save_image, **kwargs)

def _map_plot_ccube(selected_var_array, title_str, **kwargs):
    lons, lats = np.meshgrid(ESDC.country_mask.lon,ESDC.country_mask.lat)
    country_mask_2D = ESDC.country_mask[0].values
    water_mask_2D = ESDC.water_mask[0].values
    cdict = _read_ESDC_countrycodes(country_code_map)
    # reverse the key and value pair 
    idict = {v: k for k, v in cdict.items()}
    
    country_based_data_array = _expand_to_latlon(selected_var_array, country_mask_2D, idict)
    masked_country_data_array = ma.array(country_based_data_array, mask =np.isnan(country_based_data_array))
    
    vmin = kwargs["vmin"] if "vmin" in kwargs else None
    vmax = kwargs["vmax"] if "vmax" in kwargs else None
    proj = kwargs["proj"] if "proj" in kwargs else dict(projection = "kav7", lon_0=0., resolution=None)
        
    fig = plt.figure(figsize=(18,9))
    ax = fig.add_axes([0.05,0.05,0.9,0.9])
    m = Basemap(**proj)
    m.drawmapboundary(fill_color='0.3')
    ccmap = plt.cm.jet
    ccmap.set_bad("gray",1.)
    im = m.pcolormesh(lons,lats,masked_country_data_array,shading='flat',cmap=ccmap,latlon=True, vmin = vmin, vmax=vmax)
    #im2 = m.pcolormesh(lons,lats,nodata,shading='flat',cmap=plt.cm.gist_gray,latlon=True)
    m.drawparallels(np.arange(-90.,99.,30.))
    m.drawmeridians(np.arange(-180.,180.,60.))
    cb = m.colorbar(im,"bottom", size="5%", pad="2%")
    ax.set_title(title_str)
    if "save_me" in kwargs:
        if kwargs["save_me"] == True:
            plt.savefig(title_str[0:15] + '.png',dpi = 600)
    ESDC.close()
    return fig, ax, m

def _read_ESDC_countrycodes(country_code_map_csv, code_type = 'ISO3166-1-Alpha-3'):
    df = pd.read_csv(country_code_map_csv)
    cc_dict = df.set_index('ISO3166-1-numeric').to_dict()
    return cc_dict[code_type]

def _expand_to_latlon(var_array, country_mask_2D, idict):
    latlon_array = np.empty(country_mask_2D.shape)
    latlon_array.fill(np.nan)
    for country_array in var_array:
        country_code = str(country_array.Country_Codes.values)
        if country_code in idict:
            country_code_numeric = idict[country_code]
            ii = country_mask_2D == country_code_numeric
            latlon_array[ii] = country_array.values
    return latlon_array

In [ ]:
def plot_covariance(variable1_name, variable2_name, title):
    variable1 = country_cube_array.sel(Indicator_Codes = variable1_name)
    variable2 = country_cube_array.sel(Indicator_Codes = variable2_name)
    cov = get_covariance(variable1, variable2)
    _map_plot_ccube(cov, title)

def plot_corr_coeff(variable1_name, variable2_name, title):
    variable1 = country_cube_array.sel(Indicator_Codes = variable1_name)
    variable2 = country_cube_array.sel(Indicator_Codes = variable2_name)
    cov = get_covariance(variable1, variable2)
    variable1_std = variable1.std(dim="Year")
    variable2_std = variable2.std(dim="Year")
    corr_coeff = _get_corr_coeff(cov, variable1_std, variable2_std)
    _map_plot_ccube(corr_coeff, title)
    
def get_covariance(variable1_array, variable2_array):
    X = (variable1_array - variable1_array.mean(dim="Year", skipna=True))
    Y = (variable2_array - variable2_array.mean(dim="Year", skipna=True))
    xxyy = X * Y
    return xxyy.mean(dim="Year")

def _get_corr_coeff(cov, std1, std2):
    return cov / (std1 * std2)

In [ ]:
burnt_area_2010 = country_cube_array.sel(Indicator_Codes = "burnt_area",Year=2010)

In [ ]:
plot_variable_2d("evaporation", 2010, "Evaporation, ESDC, 2010", projection = MERCATOR_PROJECTION)

plot_variable_2d("SP.POP.GROW", 2010, "Annual Population Growth [%], 2010, Worldbank", vmin = -2.,vmax = 4.)

plot_variable_2d("NY.GDP.PCAP.CD", 2010, "GDP per capita [$], 2010, Worldbank", projection = ECKERT4_PROJECTION, vmin = 0.,vmax = 50000.)


In [ ]:
burnt_area = country_cube_array.sel(Indicator_Codes = "burnt_area")
pop_growth = country_cube_array.sel(Indicator_Codes = "SP.POP.GROW")
burnt_area_pop_growth_cov = get_covariance(burnt_area, pop_growth)

In [ ]:
plot_covariance("air_temperature_2m", "NY.GDP.PCAP.CD", "Covariance of air temperature and GDP per capita")

In [ ]:
plot_corr_coeff("precipitation", "NV.AGR.TOTL.ZS", "Correlation coefficient of precipitation and Agriculture, value added (% of GDP)")

In [ ]:
plot_corr_coeff("EG.USE.ELEC.KH.PC", "c_emissions", "Correlation coefficient of C Emissions and Electric power consumption (kWh per capita)")